In [1]:
import pandas as pd

In [2]:
master = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Master')
bonus = pd.read_excel('wingspan-card-list.xlsx', sheet_name='Bonus cards')

In [3]:
master = master.drop([0, 1])
master = master.rename(columns={'Unnamed: 2': 'Origin'})

In [4]:
master.to_json('master.json', orient='records')
bonus.to_json('bonus.json', orient='records')